In [2]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import seq2seq

In [3]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [4]:
len(text)

556949

In [5]:
vocabulary = set(text)
len(vocabulary)

92

In [6]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [8]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [14]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, encoder_layers = 2,
                                            decoder_hidden_units = 100, decoder_layers = 2)
net.to(device)
net.fit(source_1, target_1, verbose = 2, epochs = 10, save_path = "checkpoints/seq2seq_rnn.pt")

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 64
Out embedding dimension: 64
Encoder hidden units: 100
Encoder layers: 2
Decoder hidden units: 100
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 396,155



Epoch | Train                 | Training time
      | Loss     | Error Rate |
---------------------------------------------



    1 |   1.9521 |     49.886 |         66.94



    2 |   1.3647 |     38.564 |        133.64



    3 |   1.2147 |     34.914 |        201.72


...


    7 |   1.0039 |     29.878 |        477.84



    8 |   0.9782 |     29.245 |        549.03



    9 |   0.9567 |     28.689 |        618.41



   10 |   0.9382 |     28.224 |        688.22



,epoch,train_loss,train_error_rate,training_time,learning_rate,weight_decay,model,in_embedding_dimension,out_embedding_dimension,encoder_hidden_units,encoder_layers,decoder_hidden_units,decoder_layers,dropout,parameters
0,1,1.952125,49.886215,66.937882,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
1,2,1.364725,38.564072,133.643997,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
2,3,1.214748,34.914091,201.718107,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
3,4,1.131671,32.939692,271.644593,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
4,5,1.075995,31.638324,338.676879,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
5,6,1.035282,30.657642,406.908196,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
6,7,1.003923,29.878261,477.837474,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
7,8,0.978163,29.244913,549.031094,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
8,9,0.956667,28.689115,618.413645,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155
9,10,0.938190,28.223957,688.222672,0.0001,0,Seq2Seq RNN,64,64,100,2,100,2,0.0,396155


In [15]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100,
                                            encoder_layers = 2,
                                            decoder_hidden_units = 100,
                                            decoder_layers = 2)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
test = net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)

test[1][:20]

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 64
Out embedding dimension: 64
Encoder hidden units: 100
Encoder layers: 2
Decoder hidden units: 100
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 396,155



tensor([[ -7.2011,  -8.0435,  -8.1317,  -8.2017,  -9.3137],
        [ -6.0692,  -7.1338,  -7.5443,  -8.2454,  -8.9490],
        [ -7.4005,  -7.7818,  -8.4602,  -9.0388,  -9.2625],
        [ -7.6578,  -9.3038,  -9.6902,  -9.7532,  -9.9417],
        [ -7.4195,  -9.4202,  -9.8184, -10.0675, -10.1467],
        [ -6.6448,  -8.8351,  -9.2763,  -9.3605,  -9.5180],
        [ -6.6540,  -8.8777, -10.0707, -10.1277, -10.1403],
        [ -6.8910,  -8.5979,  -9.5578,  -9.7540,  -9.7941],
        [ -8.9615, -10.6990, -11.6517, -11.8887, -12.2937],
        [ -9.6579, -11.0449, -11.0943, -11.8142, -12.1102],
        [ -9.3646, -11.0727, -11.8167, -12.0313, -12.3229],
        [-10.6880, -11.1498, -11.6370, -12.0033, -12.2969],
        [-11.5153, -11.9960, -12.2228, -12.7509, -12.8489],
        [-10.5643, -13.4229, -13.8372, -14.0459, -14.2132],
        [-14.4716, -14.6297, -15.4080, -15.5773, -15.8926],
        [ -9.4581,  -9.5097,  -9.6156,  -9.9475, -10.8466],
        [-14.6764, -14.9207, -14.9867, -

In [77]:
importlib.reload(seq2seq)    
net = seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100,
                                            encoder_layers = 2,
                                            decoder_hidden_units = 100,
                                            decoder_layers = 2)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
test = net.sample(source_1[:1000], verbose = True, temperature = 10)

Model: Seq2Seq RNN
Tokens in the input vocabulary: 95
Tokens in the output vocabulary: 95
In embedding dimension: 64
Out embedding dimension: 64
Encoder hidden units: 100
Encoder layers: 2
Decoder hidden units: 100
Decoder layers: 2
Dropout: 0.0
Trainable parameters: 396,155

